In [1]:
import pandas as pd
import numpy as np
import string
import preprocessor as p
import re
from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia=SIA()

In [10]:
# 전처리 함수 내 사용
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "&gt;" : "is greater than", "&lt;" : "is less than", "&nbsp;": " ", "&amp;" : "and", "&quot;" : '"'}

# 영어 사진
words = set(nltk.corpus.words.words())

def preprocessing(data) :
    data = re.sub('’|‘|´|`', '\'', data)
    data = re.sub('\\n', ' ', data)
    # lowering
    data = data.lower()
    # normalize abbreviation
    data = ' '.join([contractions[t] if t in contractions else t for t in data.split(" ")])
    # clean corpus to remove emoji and URL
    data = p.clean(data)
    # remove words shorter than length 1
    data = ' '.join(word for word in data.split())
    return data

def only_alpha(data) :
    # remove numbers and punctuations
    data = re.sub(r'[^A-Za-z ]', ' ', data)
    # remove duplicated alphabets
    data = re.sub(r'([A-Za-z])\1{2,}', r'\1', data)
    # remove words shorter than length 1
    data = ' '.join(word for word in data.split() if len(word) > 1)
    return data

def approx_english_detection(line) :
    total_words = line.split()
    eng_num = 0;
    for w in total_words: 
        if w in words : eng_num += 1
    if(eng_num / len(total_words) >= 0.5) : return 'en'
    return 'other'

def get_score(line) :
    score = sia.polarity_scores(line)['compound']
    if score > 0.2 : return 1
    elif score < -0.2 : return -1
    return 0

In [76]:
keyword = input("keyword : ")

keyword : pubg


In [92]:
filename = input("file : ")

file : pubg_2022-04-13_2022-05-16_tweetdata


In [93]:
df = pd.read_csv("./corpus/" + keyword + '/' + filename + ".csv", encoding='UTF-8')
df = df[['Datetime','Text']]

# 전처리
df['Text'] = df['Text'].apply(lambda x : preprocessing(x))
df_temp = df[df['Text'] == ''].index
df = df.drop(df_temp)

# 영어 사진에 키워드 추가
words.add(keyword)

df['approx'] = df['Text'].apply(lambda x : approx_english_detection(x))
result = df.loc[df['approx'] == 'en']

result['Text'] = result['Text'].apply(lambda x : only_alpha(x))

# 불용어 처리
stop_words = set(stopwords.words('english')) 
result['Clean'] = result['Text'].apply(lambda x : ' '.join(word for word in x.split() if not word in stop_words if len(word) > 1))

# 없어진 문장 처리
result_temp = result[result['Clean'] == ''].index
result = result.drop(result_temp)

result = result[['Datetime', 'Text', 'Clean']].set_index('Datetime')
result.to_csv('./preprocessed/' + keyword + '/' + filename + '_processed_revised.csv', encoding='UTF-8')

C:\Users\82106\AppData\Local\Temp/ipykernel_19696/542986543.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Text'] = result['Text'].apply(lambda x : only_alpha(x))
C:\Users\82106\AppData\Local\Temp/ipykernel_19696/542986543.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Clean'] = result['Text'].apply(lambda x : ' '.join(word for word in x.split() if not word in stop_words if len(word) > 1))


In [94]:
result['Label'] = result['Text'].apply(lambda x : get_score(x))
result.to_csv('./labeled/' + keyword + '/revised/' + filename + '_labeled_revised.csv', encoding='UTF-8')

In [95]:
positive = result[result['Label'] == 1]
neutral = result[result['Label'] == 0]
negative = result[result['Label'] == -1]

In [96]:
positive.to_csv("./labeled/"+keyword+'/revised/'+filename+'_positive_revised.csv')
negative.to_csv("./labeled/"+keyword+'/revised/'+filename+'_negative_revised.csv')
neutral.to_csv('./labeled/' +keyword+'/revised/'+filename+'_neutral_revised.csv')